"Talk to Your Database" (Text-to-SQL Bot) 🔥 (Highly Recommended)
Abhi tumne unstructured data (text/articles) ke sath chat ki. Industry mein asli paisa Structured Data (SQL Databases) mein hai. Companies chahti hain ke unka manager seedha puche: "Pichle mahene kitni sales hui?" aur bot database se query kar ke jawab de.
    • Concept: User English mein sawal puchega -> Bot usay SQL Query mein convert karega -> Database par run karega -> Result ko wapis English mein batayega.
    • Kya seekho ge:
        ○ LangChain SQL Toolkit: Database se kaise connect karte hain.
        ○ Prompt Engineering for Code: LLM se accurate SQL kaise likhwana hai.
        ○ Security: Injection attacks se kaise bachen.
Industry Value: Har company ke paas database hai, lekin har manager ko SQL nahi aati. Yeh tool unke liye magic hai.

In [1]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from langchain_classic.storage import InMemoryStore
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_classic.retrievers import ParentDocumentRetriever
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_community.utilities import SQLDatabase
# Corrected import for create_sql_query_chain
from langchain_classic.chains import create_sql_query_chain
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("HUGGINGFACE_API_KEY")

str_parser = StrOutputParser()

C:\Users\Dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [14]:
question = input("What is your question? ")

In [17]:
llm = HuggingFaceEndpoint(
    # repo_id="deepseek-ai/DeepSeek-R1",
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    task="text-generation",
    huggingfacehub_api_token=api_key,
)

model = ChatHuggingFace(llm=llm, temperature=0.9)

In [4]:


db_user = "root"          # 1. Yahan apna MySQL username likhein (default: root)
db_password = ""          # 2. Yahan apna password likhein (agar koi hai to)
db_host = "localhost"     # 3. Agar database isi computer pe hai to 'localhost' likhein
db_name = "ok"     # 4. YAHAN APNI DATABASE KA NAAM LIKHEIN (Yehi "path" hai)

# Connection String
mysql_uri = f"mysql+pymysql://{db_user}:{db_password}@{db_host}:3306/{db_name}"

try:
    db = SQLDatabase.from_uri(mysql_uri)
    print("✅ Database Connected Successfully!")
    print("Dialect:", db.dialect)
    print("Tables found:", db.get_usable_table_names())
except Exception as e:
    print("❌ Connection Error! Check your details.")
    print(e)

✅ Database Connected Successfully!
Dialect: mysql
Tables found: ['cache', 'cache_locks', 'categories', 'company_settings', 'customer_advances', 'customer_credit_payments', 'customers', 'expense_categories', 'expenses', 'failed_jobs', 'job_batches', 'jobs', 'migrations', 'panaflex_specs', 'password_reset_tokens', 'payments', 'pending_payments', 'permissions', 'products', 'purchase_items', 'purchases', 'register_sessions', 'role_permissions', 'roles', 'sale_items', 'sale_return_items', 'sale_returns', 'sales', 'sessions', 'stock_adjustments', 'stock_batches', 'stock_moves', 'suppliers', 'units', 'user_permissions', 'users']


In [5]:
primary_prompt = PromptTemplate(
    template  = """You are a senior SQL expert and database analyst.

Your task is to generate a syntactically correct, optimized SQL query
based ONLY on the provided database schema.

STRICT RULES:

1. Use ONLY tables and columns mentioned in the schema.
2. Do NOT assume columns that are not listed.
3. If question is ambiguous, choose the most logical interpretation.
4. Always prefer aggregation functions when user asks for totals, averages, counts.
5. Use proper JOIN conditions if multiple tables are required.
6. Do NOT generate explanations.
7. Return ONLY SQL query.
8. Use LIMIT 10 if user asks for listing data without specifying limit.
9. Always handle date-based filtering carefully.
10. If query cannot be answered using schema, return:
   -- CANNOT_GENERATE_SQL

DATABASE SCHEMA:
{schema}

USER QUESTION:
{question}

SQL QUERY:
""",
    input_variables=["schema", "question"],
)

schema_ext_prompt = PromptTemplate(
    template="""You are a professional data analyst.

You are given:
- User Question
- SQL Query
- SQL Result

Convert the SQL result into a clear, human-readable answer.

RULES:
1. Be concise.
2. Use simple business language.
3. If result is empty, say:
   "No data found based on current records."
4. If numeric result, clearly mention the value.
5. Do NOT mention SQL query in final answer.

USER QUESTION:
{question}

SQL QUERY:
{query}

SQL RESULT:
{result}

FINAL ANSWER:
""",
    input_variables=["question", "query", "result"],
)

In [6]:

# # Create the Text-to-SQL Chain
# chain = create_sql_query_chain(model, db)

# # 4. Test the chain
# # Note: Ensure the 'question' variable asks about the 'users' table, e.g., "How many users are from Lahore?"
# print(f"Processing question: {question}")
# response = chain.invoke({"question": question})
# print("Generated SQL:", response)

# # 5. Execute the query
# # To clean up the SQL output from the LLM (sometimes it includes markdown), you might need additional parsing.
# # For simple cases:
# clean_sql = response.strip().replace("```sql", "").replace("```", "")
# try:
#     result = db.run(clean_sql)
#     print("Query Result:", result)
# except Exception as e:
#     print("Error executing SQL:", e)

In [ ]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

# Toolkit banayein (database tools)
toolkit = SQLDatabaseToolkit(db=db, llm=model)

# Agent banayein
agent = create_sql_agent(
    llm=model,
    toolkit=toolkit,
    verbose=True,  # Debugging ke liye - dikhayega kya ho raha hai
    agent_type="zero-shot-react-description",  # LangChain ka default SQL agent type
    handle_parsing_errors=True  # Agar LLM galat format de to agent ko retry ka mauka milta hai
)

In [18]:
response = agent.invoke({"input": question})
print(response["output"])



> Entering new SQL Agent Executor chain...
<think>
We are given a question about "sale revenue of January month 2026"
 Steps:
 1. First, we need to know what tables are available in the database.
 2. Then, we need to find which table(s) contain sales data and revenue information, and also a date or timestamp to filter by January 2026.
 3. We'll check the schema of the relevant tables to see the columns and data types.
 4. Construct a SQL query that:
    - Selects the revenue (or calculates it if necessary)
    - Filters for records in January 2026 (i.e., between '2026-01-01' and '2026-01-31')
    - Sums up the revenue for that period (if the question is about total revenue) or lists individual records (if the question is about transactions, but the question asks for "sale revenue" which likely means total).
 5. Since the question asks for the revenue of the entire month, we should aggregate the revenue.

 However, note: the question does not specify if it's total revenue or per some 

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Parsing LLM output produced both a final answer and a parse-able action:: <think>
We encountered an error because the input to `sql_db_schema` was not correctly formatted. The tool expects a comma-separated list of table names. Let me try again by providing the table names as a comma-separated string without any extra text.

Action: sql_db_schema
Action Input: sales, sale_items
</think>I see that the sales table has columns like "total" and "created_at", which are exactly what I need. The total column likely represents the revenue for each sale, and created_at can be used to filter for January 2026.

Now I can construct the SQL query to calculate the total revenue for January 2026 by summing the "total" column for all sales that occurred in that month.

Action: sql_db_query_checker  
Action Input: S

In [7]:
def get_schema(unused_input):
    # 'db' aapka pehle se bana hua database object hai
    return db.get_table_info()

print(db.get_usable_table_names())
# print(db._get_table_indexes("users"))

['cache', 'cache_locks', 'categories', 'company_settings', 'customer_advances', 'customer_credit_payments', 'customers', 'expense_categories', 'expenses', 'failed_jobs', 'job_batches', 'jobs', 'migrations', 'panaflex_specs', 'password_reset_tokens', 'payments', 'pending_payments', 'permissions', 'products', 'purchase_items', 'purchases', 'register_sessions', 'role_permissions', 'roles', 'sale_items', 'sale_return_items', 'sale_returns', 'sales', 'sessions', 'stock_adjustments', 'stock_batches', 'stock_moves', 'suppliers', 'units', 'user_permissions', 'users']


In [8]:
parallel_chain = RunnableParallel(
    {"schema": RunnableLambda(get_schema) | model,
     "question": RunnablePassthrough(),
     "result": schema_ext_prompt | model
     }
)

In [9]:
chain_1 = (
    RunnableLambda(get_schema) | model
)
result_1 = chain_1.invoke(db)
print(result_1)

content='<think>\nWe are given the database schema and sample data for a point of sale system for a printing shop.\n We are to generate a query that retrieves the total sales for each customer for the month of January 2026, \n including the customer\'s name, total sales amount, and the number of transactions.\n\n Steps:\n 1. We note that sales are stored in the `sales` table. The relevant columns are:\n    - `id`: sale identifier\n    - `customer_id`: references the `customers` table\n    - `grand_total`: the total amount of the sale\n    - `sold_at`: the datetime of the sale\n\n 2. We are interested in sales that occurred in January 2026. We can filter by:\n    - `sold_at` between \'2026-01-01\' and \'2026-01-31\'\n\n 3. We need to join with the `customers` table to get the customer name.\n\n 4. We note that there are opening balance entries in the sales table (with invoice_no starting with \'OPB-\'). \n    We should exclude these because they are not sales transactions? \n    However

In [10]:
sql_chain = (
    RunnableParallel({
        "schema": RunnableLambda(get_schema),
        "question": RunnablePassthrough()
    })
    | primary_prompt 
    | model 
    | str_parser
)

result_2 = sql_chain.invoke({"question": question})

print(result_2)

<think>
Okay, let's tackle this user question: "What is sale revenue of this month?" 

First, I need to understand what the user is asking. Revenue typically refers to the total income from sales. So, they likely want the sum of all sales amounts for the current month.

Looking at the database schema, the relevant table is 'sales'. The 'sales' table has a 'grand_total' column which probably represents the total amount of each sale. There's also a 'sold_at' column which is a DATETIME, so that can be used to filter sales from the current month.

The steps I need to take are:
1. Sum the 'grand_total' from all sales where 'sold_at' falls within the current month.
2. Use the MONTH() and YEAR() functions to check if the 'sold_at' date is in the current month and year.
3. Use the SUM() aggregation function to get the total revenue.

I should also consider if there are any refunds or returns that might affect the revenue. But the user didn't mention returns, and the 'sale_returns' table exists

In [11]:
# Chain Definition
chain = (
    parallel_chain | primary_prompt | model | str_parser
)

# Invoke Chain
# Note: Input Dictionary mein sirf woh cheez pass kartay hain jo 'start' mein chahye (user ka sawal).
# 'query', 'result' aur 'schema' keys parallel_chain KHUD generate karega. Use bahar se pass nahi karna.
print(f"User Question: {question}")
result = chain.invoke(question) # Sirf question string pass karein
print("Final Answer:", result)

User Question: What is sale reveue of this month?


KeyboardInterrupt: 